In [16]:

def _pad_csr(A, row_scale, col_scale, insertrow=0, insertcol=0):
    """
    Expand the input csr_matrix to a greater space as given by the scale.
    Effectively inserting A into a larger matrix
         zeros([A.shape[0]*row_scale, A.shape[1]*col_scale]
    at the position [A.shape[0]*insertrow, A.shape[1]*insertcol]
    The same could be achieved through using a kron with a matrix with
    one element set to 1. However, this is more efficient
    """

    # ajgpitch 2016-03-08:
    # Clearly this is a very simple operation in dense matrices
    # It seems strange that there is nothing equivalent in sparse however,
    # after much searching most threads suggest directly addressing
    # the underlying arrays, as done here.
    # This certainly proved more efficient than other methods such as stacking
    #TODO: Perhaps cythonize and move to spmatfuncs

    if not isinstance(A, sp.csr_matrix):
        raise TypeError("First parameter must be a csr matrix")
    nrowin = A.shape[0]
    ncolin = A.shape[1]
    nrowout = nrowin*row_scale
    ncolout = ncolin*col_scale

    A._shape = (nrowout, ncolout)
    if insertcol == 0:
        pass
    elif insertcol > 0 and insertcol < col_scale:
        A.indices = A.indices + insertcol*ncolin
    else:
        raise ValueError("insertcol must be >= 0 and < col_scale")

    if insertrow == 0:
        A.indptr = np.concatenate((A.indptr,
                        np.array([A.indptr[-1]]*(row_scale-1)*nrowin)))
    elif insertrow == row_scale-1:
        A.indptr = np.concatenate((np.array([0]*(row_scale - 1)*nrowin),
                                   A.indptr))
    elif insertrow > 0 and insertrow < row_scale - 1:
         A.indptr = np.concatenate((np.array([0]*insertrow*nrowin), A.indptr,
                np.array([A.indptr[-1]]*(row_scale - insertrow - 1)*nrowin)))
    else:
        raise ValueError("insertrow must be >= 0 and < row_scale")

    return A

In [17]:
%timeit 1+2

4.33 ns ± 0.00679 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [18]:
import scipy.sparse as sp
import numpy as np

In [19]:
from qutip import *

In [50]:
Q = destroy(100)

N_he = 10
block =  100*100


op = spre(Q).data

In [8]:
st

<function time.time>

In [51]:
block = 100*100
import time as time
L_helems = sp.lil_matrix(
            (N_he*block,N_he*block),
            dtype=np.complex,
        )

st= time.time()
L_helems[0 : 0 + block, 0 : 0 + block] += op
end = time.time()

print(end-st)

10.784727334976196


In [52]:

L_helems2 = sp.csr_matrix((N_he*block, N_he*block),
                                     dtype=complex)

st= time.time()
L_helems2 = _pad_csr(op, N_he, N_he, 0, 0)
end = time.time()

print(end-st)

0.004386186599731445


In [53]:
L_helems.tocsr().data


array([1.        +0.j, 1.41421356+0.j, 1.73205081+0.j, ...,
       9.8488578 +0.j, 9.89949494+0.j, 9.94987437+0.j])

In [54]:
L_helems2.data

array([1.        +0.j, 1.41421356+0.j, 1.73205081+0.j, ...,
       9.8488578 +0.j, 9.89949494+0.j, 9.94987437+0.j])

In [55]:
L_helems - L_helems2

<100000x100000 sparse matrix of type '<class 'numpy.complex128'>'
	with 0 stored elements in Compressed Sparse Row format>